In [7]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import dash_table
import pandas as pd



In [8]:
# Build App
app = JupyterDash(__name__,suppress_callback_exceptions=True)

#function to query dbpedia endpoint
def sql(value):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(value)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert() #is a JSON dict
    
    cols = results['head']['vars']

    
    #file = list() #create an empty list
    #loop to put each of the individual records in a new row
    #for result in results:
    #    file.append(result)        
    #return file
    
    return results

app.layout = html.Div([
    
    html.Div(
        className="navbar",
        children=[ 
            html.H3(
                children="Interactive Data Analytics Dashboard for RDF Knowledge Graphs", 
                className="navbar--title"),
            html.Div(
                children=[
                dcc.Tabs(id='tabs', value='tab-1', children=[
                    dcc.Tab(label='Dashboard', value='Dashboard'),
                    dcc.Tab(label='Tables', value='Tables'),
                    dcc.Tab(label='Charts', value='Charts'),
                    dcc.Tab(label='Query Samples', value='Samples'),
                    dcc.Tab(label='Upload', value='Upload'),
                    dcc.Tab(label='Download', value='Download'),
                ]),
                html.Div(id='tabs-content')
                ])
    ]),
   
       
])#lay-out div    

#Query examples
#data = {'Query Examples':  ['First example', 'Second example','third example']}
#df = pd.DataFrame (data, columns = ['Query Examples'])

data = {'Query Examples':  ['Homepages of Tom Cruise', 'Second example','third example'], 'content': ['some info about Tom', 'b','c']}
df = pd.DataFrame (data, columns = ['Query Examples', 'content'])

print(df)

@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
)

def render_content(tab):
    if tab == 'Dashboard':
        return html.Div([
            html.Div(
                className="querybox",
                children=[
                    html.H4(
                        className="querybox--title",
                        children="SPARQL Query"
                    ),
                    dcc.Textarea(
                        id= "query-text", 
                        value="Enter your SPARQL query.", 
                        className="querybox--textarea"
                    ),
                    html.Button(
                        id="submit-btn", className="querybox--btn", 
                        children="SUBMIT", 
                        n_clicks=0
                    )
            ]),
            html.Div(
                id="output",
                className="output"
            )
             
        ])
    elif tab == 'Tables':
        return html.Div([
            html.H3('Tables')
        ])
    elif tab == 'Charts':
        return html.Div([
            html.H3('Charts')
        ])
    elif tab == 'Samples':
        return html.Div([
            html.H3('Query Samples') ,
            html.Div(
                className="querybox",
                children=[
                    html.H4(
                        className="querybox--title",
                        children="SPARQL Query"
                    ),
                    dcc.Textarea(
                        id= "query-text", 
                        value="Enter your SPARQL query.", 
                        className="querybox--textarea"
                    ),
                    html.Button(
                        id="submit-btn", className="querybox--btn", 
                        children="SUBMIT", 
                        n_clicks=0
                    )
            ]),
            html.Div(
                className="examples",
                children=[
                    dash_table.DataTable(
                    id='examples_table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    style_table={'width': '800px'},
                    #row_selectable='single',
                    #row_deletable=False,
                    style_cell=dict(textAlign='left'), 
                   
                    )
                ]),
            html.Div(
                id="examples_table_container",
            )
        ])
    
@app.callback(
    Output("output", "children"),
    Input("submit-btn", "n_clicks"),
    State("query-text","value")
)    
def update_output(n_clicks, value):
    if n_clicks > 0:
        resultList = sql(value)
       # resultListDecoded = list()
       # for result in resultList:   
       #     resultListDecoded.append(result.decode("utf-8"))
        
       # return resultListDecoded
    
    
@app.callback(
    Output("examples_table_container", "children"),
    Input("examples_table", 'active_cell')   
)
def selected_example(active_cell):
        print(active_cell)
        active_row_id = active_cell['row'] if active_cell else None
        print(active_row_id)
       #Then we need to grab the content of the query example and 

            Query Examples              content
0  Homepages of Tom Cruise  some info about Tom
1           Second example                    b
2            third example                    c


In [9]:
# Run app and display result external in the notebook
app.run_server(mode='external', port=8051 )

Dash app running on http://127.0.0.1:8051/
